# 모델 완성

In [ ]:
import time
import logging
import traceback
import pandas as pd

df_추천clean = pd.read_csv('추천숙소정보0829.csv')
df_최종프레임 = pd.read_csv('최종_추천시스템0828.csv')

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s"
)

# =========================
# Gemini: 루프 바깥에서 1회 초기화
# =========================
import google.generativeai as genai

API_KEY = "gemini_api_key"  # 발급받은 실제 키
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel('gemini-2.0-flash-001')  # 이미지 입력 지원 모델


# =========================
# 그대로 유지: 데이터 가져오기
# =========================
def get_df_selections():
    import requests
    import pandas as pd
    import random
    import hashlib

    WIX_API_URL = "wix_url"
    query_params = {"limit": "5"}
    response = requests.get(WIX_API_URL, params=query_params, timeout=20)
    response.raise_for_status()
    result_json = response.json()

    selection_items = result_json.get("items", [])
    next_cursor = result_json.get("nextCursor")

    # DataFrame 생성
    df_selections = pd.DataFrame(selection_items)

    # 필요한 컬럼만 추출 및 순서 지정
    selection_columns = ["email", "region", "tone", "style", "view", "leisure"]
    df_selections = df_selections[selection_columns]

    # ---------------------- 추가 부분 ----------------------
    # 추천시스템.csv 불러오기 (작성자 ID 활용)
    df_추천DB = df_최종프레임.copy()

    # 이메일 해시값을 기반으로 항상 같은 작성자 ID를 반환하는 함수
    def assign_fixed_author(email, df_authors):
        author_list = df_authors['작성자'].unique()
        seed = int(hashlib.sha256(str(email).encode('utf-8')).hexdigest(), 16)
        idx = seed % len(author_list)
        return author_list[idx]

    # df_selections에 가상 작성자 매핑 추가
    df_selections['작성자'] = df_selections['email'].apply(
        lambda x: assign_fixed_author(x, df_추천DB)
    )
    # -------------------------------------------------------

    # # CSV 저장
    df_selections.to_csv("wix_selections.csv", index=False, encoding="utf-8-sig")

    return df_selections


# =========================================
# 20초 주기 무한 실행 + 0번 인덱스 변화 감지
# =========================================
WATCH_COLS = ["email", "region", "tone", "style", "view"]
last_signature = None  # (email, region, tone, style, view) 튜플 저장

while True:
    try:
        df = get_df_selections()

        if df.empty:
            logging.info("빈 df_selections. 20초 후 재시도.")
        else:
            # 0번 인덱스 기준 현재 시그니처
            current_signature = tuple(df.loc[0, WATCH_COLS].astype(str).tolist())

            if last_signature is None:
                last_signature = current_signature
                logging.info(f"기준 시그니처 설정: {last_signature}")

            elif current_signature != last_signature:
                logging.info(f"변화 감지! 이전: {last_signature} → 현재: {current_signature}")

                # =========================
                # 1차 필터링
                # =========================
                import joblib

                loaded_svd = joblib.load('best_svd_model.joblib')

                def recommend_top_n(df_selections, loaded_svd, user_index, n=400):
                    user_id = df_selections.iloc[user_index]['작성자']
                    rated_accommodations = df_최종프레임.loc[df_최종프레임['작성자'] == user_id, '숙소ID'].tolist()
                    all_accommodations = df_최종프레임['숙소ID'].unique()
                    unrated_accommodations = [item for item in all_accommodations if item not in rated_accommodations]

                    predictions = []
                    for accommodation_id in unrated_accommodations:
                        predicted_rating = loaded_svd.predict(str(user_id), str(accommodation_id))
                        predictions.append(predicted_rating)

                    predictions.sort(key=lambda x: x.est, reverse=True)
                    top_n_recommendations = predictions[:n]

                    print(f"\n{user_id} 님에게 추천하는 숙소 TOP {n}:")
                    for i, pred in enumerate(top_n_recommendations):
                        print(f"{i+1}위: 숙소ID {pred.iid} - 예상 평점: {pred.est}")

                    top_n_ids = [str(p.iid) for p in top_n_recommendations]
                    score_map = {str(p.iid): float(p.est) for p in top_n_recommendations}
                    return top_n_ids, score_map

                top_n_ids, score_map = recommend_top_n(df, loaded_svd, user_index=0, n=400)

                recommend_df = (
                    df_추천clean.copy()
                    .assign(숙소ID=lambda d: d['숙소ID'].astype(str))
                    .set_index('숙소ID')
                    .reindex(top_n_ids)
                    .reset_index()
                )
                recommend_df['예상평점'] = recommend_df['숙소ID'].map(score_map)

                result1 = recommend_df
                # =========================
                # 1차 필터링 끝
                # =========================

                # =========================
                # 2차 필터링
                # =========================
                import re, json
                import pandas as pd

                SIDO_CANON = ["서울","경기","인천","강원","부산","대구","광주","대전","울산","세종",
                              "충북","충남","전북","전남","경북","경남","제주"]

                def normalize_sido_token(x: str) -> str:
                    if x is None:
                        return ""
                    s = str(x).strip()
                    s = s.split()[0]
                    s = (s.replace("특별자치도","").replace("광역시","").replace("특별시","")
                         .replace("자치시","").replace("자치도","").replace("도","").replace("시",""))
                    mapping_prefix = {
                        "충청북":"충북","충청남":"충남","전라북":"전북","전라남":"전남",
                        "경상북":"경북","경상남":"경남","제주특별자치":"제주"
                    }
                    for k,v in mapping_prefix.items():
                        if s.startswith(k):
                            s = v
                    return s

                def chat_with_gemini_text(model, prompt: str) -> str:
                    resp = model.generate_content(prompt)
                    return (resp.text or "").strip()

                def build_prompt_for_region(user_region: str) -> str:
                    return f"""
                다음 입력을 한국 '시/도'로 분류해.
                입력: "{user_region}"

                반드시 아래 후보 중에서만 선택해라:
                {", ".join(SIDO_CANON)}

                출력 형식은 아래 JSON **그대로**만 허용한다.
                예) {{"sido": ["경기"]}}    또는    {{"sido": ["경기","강원"]}}  (최대 3개)
                """

                def classify_sido_with_gemini(model, user_region: str) -> list:
                    prompt = build_prompt_for_region(user_region)
                    try:
                        raw = chat_with_gemini_text(model, prompt)
                        data = json.loads(raw)
                        lst = [t for t in data.get("sido", []) if t in SIDO_CANON]
                        return list(dict.fromkeys(lst)) or SIDO_CANON[:]
                    except Exception:
                        return SIDO_CANON[:]

                user_region = str(df.iloc[0]["region"])
                sido_choices = classify_sido_with_gemini(model, user_region)

                recommend_df = result1.copy()
                recommend_df["표준시도"] = recommend_df["시도"].apply(normalize_sido_token)

                filtered = (
                    recommend_df[recommend_df["표준시도"].isin(sido_choices)]
                    .reset_index(drop=True)
                )

                print("입력 region:", user_region)
                print("Gemini 선택(시/도):", sido_choices)
                print(f"2차 필터 결과: {len(filtered)} rows")

                result2 = filtered
                # =========================
                # 2차 필터링 끝
                # =========================

                # =========================
                # 3차 필터링
                # =========================
                def filter_recommend_by_user(df_selections, filtered, user_index=1):
                    df_selections = df_selections.rename(columns=lambda x: x.lower())
                    filtered = filtered.rename(columns=lambda x: x.lower())

                    user_row = df_selections.iloc[user_index]
                    user_tone = str(user_row['tone']).lower()
                    user_style = str(user_row['style']).lower()
                    user_view = str(user_row['view']).lower()
                    
                    filtered = filtered[
                        (filtered['color'].str.lower() == user_tone) |
                        (filtered['style'].str.lower() == user_style) |
                        (filtered['view'].str.lower() == user_view)
                    ]
                    return filtered

                result3 = filter_recommend_by_user(df, result2, user_index=1)
                print(f"3차 필터 결과: {len(result3)} rows")
                # =========================
                # 3차 필터링 끝
                # =========================

                # =========================
                # 최종 업로드
                # =========================
                try:
                    import requests
                    BASE = "wix_url"
                    KEY  = "demo"

                    result_df = result3.copy()

                    safe = (
                        result_df.head(6)
                        .assign(
                            숙소명=lambda d: d["숙소명"].astype(str),
                            숙소url=lambda d: d["숙소url"].astype(str),
                            gsrm_image_url=lambda d: d["gsrm_image_url"].astype(str),
                        )
                    )

                    items = [
                        {"title": r["숙소명"], "url": r["숙소url"], "imageUrl": r["gsrm_image_url"]}
                        for _, r in safe.iterrows()
                    ]

                    if items:  # 비어있지 않을 때만 전송
                        res = requests.post(
                            f"{BASE}/gallerydata",
                            json={"key": KEY, "items": items},
                            timeout=20
                        )
                        res.raise_for_status()
                        print("Wix 업로드 OK:", res.json())
                    else:
                        logging.info("업로드할 items 없음")
                        # --- 추가 부분: 무결과도 빈 items 업로드 ---
                        try:
                            res = requests.post(
                                f"{BASE}/gallerydata",
                                json={"key": KEY, "items": []},
                                timeout=20
                            )
                            res.raise_for_status()
                            print("Wix 업로드(무결과):", res.json())
                        except Exception:
                            logging.exception("Wix 업로드 실패(무결과)")

                except Exception:
                    logging.exception("Wix 업로드 실패")

                last_signature = current_signature

            else:
                logging.debug("변화 없음.")

    except Exception as e:
        logging.exception("주기 실행 중 오류 발생 (계속 진행):")

    time.sleep(20)
